In [1]:
# Lost Cities V1E
# Starting with V1C
# Creating separate main file and capturing all parameters together
# Still using 43 features

In [3]:
# import V1E_main as main
from V1E_main import *

In [4]:
# dir(main)

In [7]:
# ALL PARAMETERS
# File version
fv='V1E_2'
file_name='all_rewards.'+fv+'.csv'

nn_layer_1=96
nn_layer_2=32
nn_layer_2_dropout=0.20
learning_rate=0.001
replay_size=18000
num_episodes = 60_000
batch_size = 64
batch_cnt = 3
train_every = 5
step_booster = 5.0
episode_booster = 0.10 # 0.5 for 1, 1.0 for 2, 0.0 for 3
all_rewards = []
mean_rewards = []
epsilon = 0.35
epsilon_min = 0.030 # was 0.35 for .2.
epsilon_decay = 0.999999  # adjust this rate as needed - 0.99995 is too low

# Simply comment out any functions not to be included
step_functions=[
'lover_val_avail',
'too_few_pts',
'blocked_7',
'exp_small_deck',
'exp_was_live',
'good_exp',
'bad_X',
'bad_bigger_val',
'good_low_val',
'draw_to_tgt',
# 'had_X',
'next_value',
# 'bad_center',
# 'smart_opp_center'
]

In [9]:
config_dict = {
    "nn_layer_1": nn_layer_1,
    "nn_layer_2": nn_layer_2,
    "nn_layer_2_dropout": nn_layer_2_dropout,
    "learning_rate": learning_rate,
    "replay_size": replay_size,
    "num_episodes": num_episodes,
    "batch_size": batch_size,
    "batch_cnt": batch_cnt,
    "train_every": train_every,
    "step_booster": step_booster,
    "episode_booster": episode_booster,
    "epsilon": epsilon,
    "epsilon_min": epsilon_min,
    "epsilon_decay": epsilon_decay
}

save_config_txt(fv, config_dict, step_functions)

Configuration saved to config_V1E_2.txt


In [11]:
# Modified Training Loop with Action + Draw Selection from Policy
env = LostCitiesEnv()
num_card_actions = card_cnt
num_draw_choices = color_cnt+1
model = ActorCritic(state_size=state_size, action_size=num_card_actions, draw_size=num_draw_choices, 
                         nn_layer_1=nn_layer_1, nn_layer_2=nn_layer_2, nn_layer_2_dropout=nn_layer_2_dropout)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
replay_buffer = deque(maxlen=replay_size)
rule_counter = defaultdict(int)

for episode in range(1, num_episodes + 1):
    state = env.reset()
    done = False
    mean_reward = 0.0
    play_cnt=0
    plays_p1 = []
    plays_p2 = []

    while not done:
        play_cnt+=1
        features_np = extract_features(state)
        features = torch.tensor(features_np, dtype=torch.float32)

        # Forward pass through the model
        card_logits, draw_logits, value = model(features)

        # Get legal actions and legal draws
        # actions, draws = env.get_legal_actions(state['current_player'])
        actions, draws = env.get_legal_actions(state['current_player'])
        legal_action_indices = list(range(len(actions)))

        valid_draws = [d for d in draws if d == 'deck' or (d in env.center_piles and env.center_piles[d])]
        if not valid_draws:
            print("No valid draws—forcing episode end.")
            done = True
            break

        # legal_draw_indices = list(range(len(valid_draws)))  # typically ['deck', 'R', 'B', 'G']

        # Stop this game is no legal actions - though this is of course not true, but...
        # Print discard and draw for plays 1001 to 1020
        if 10001 <= play_cnt <= 10020:
            print(f"Legal actions: {actions}")
            print(f"Action indices: {[i for i in range(len(actions))]}")
            # Not valid at this point
            # print(f"Play {play_cnt}: Discard action = {chosen_action}, Draw choice = {chosen_draw}")
        if play_cnt>=10020:
            print(play_cnt, actions, draws, valid_draws)
            print(f"\n--- STUCK STATE at play {play_cnt} ---")
            print(f"Deck size: {state['deck_size']}")
            print(f"Player hand: {state['hands'][state['current_player']]}")
            print(f"Expeditions:")
            for color in env.expeditions[state['current_player']]:
                print(f"  {color}: {env.expeditions[state['current_player']][color]}")
            print(f"Center piles:")
            for color in env.center_piles:
                print(f"  {color}: {env.center_piles[color]}")
            print(f"Available actions: {actions}")
            print(f"Available draws: {draws}")
            print(f"----------------------\n")
            raise SystemExit(f"STOP")
        
        if not actions:
            print(f"No legal actions for player {state['current_player']}. Ending episode early.")
            done = True
            break

        # Sample card action with epsilon-greedy
        if random.random() < epsilon:
            # Random action
            card_idx = random.randint(0, len(actions) - 1)
        else:
            # Model-based action
            card_probs = torch.softmax(card_logits[:len(actions)], dim=0)
            card_dist = torch.distributions.Categorical(card_probs)
            card_idx = card_dist.sample().item()
        
        chosen_action = actions[card_idx]

        # Filter valid draws based on chosen_action (if it's a center discard)
        discard_color = None
        if chosen_action[0] == 'center':
            discard_color = chosen_action[1][0]
        
        filtered_draws = [
            d for d in valid_draws if d != discard_color
        ]
        if not filtered_draws:
            # Failsafe: fallback to deck
            filtered_draws = ['deck']

        valid_draws=filtered_draws

        # Sample draw choice (FIXED)
        if random.random() < epsilon:
            chosen_draw = random.choice(valid_draws)
        else:
            # Correct mapping: get logits only for valid draws
            draw_indices_in_logits = [draw_to_index[d] for d in valid_draws]
            draw_logits_filtered = draw_logits[draw_indices_in_logits]
            draw_probs = torch.softmax(draw_logits_filtered, dim=0)
            draw_dist = torch.distributions.Categorical(draw_probs)
            draw_idx = draw_dist.sample().item()
            chosen_draw = valid_draws[draw_idx]

        # Compute shaped intermediate reward
        step_reward = compute_step_reward(state, chosen_action, chosen_draw, env, step_functions, rule_counter)

        # Map draw_choice to its index for policy update
        chosen_draw_idx = draw_to_index[chosen_draw]

        # Save the current player before doing env.step
        current_player=state['current_player']
        
        # Take action and draw based on policies
        next_state, reward, done = env.step(chosen_action, chosen_draw)

        # Combine shaped reward + final score (if any)
        # total_reward = reward + booster * step_reward
        total_reward = step_booster * step_reward

        # Store full experience (must include both action idx and draw idx!)
        # replay_buffer.append((features_np, card_idx, chosen_draw_idx, total_reward))
        # Now, do it all at end of game
        if current_player=='P1':
            plays_p1.append((features_np, card_idx, chosen_draw_idx, step_reward))
        else:
            plays_p2.append((features_np, card_idx, chosen_draw_idx, step_reward))
            
        # Advance state
        state = next_state
        mean_reward += total_reward

        # Annealing
        epsilon = max(epsilon_min, epsilon * epsilon_decay)

        ddebug = random.random()<1e-6
        if done:
            reward_p1 = env.compute_score('P1')
            reward_p2 = env.compute_score('P2')
            p1cnt=0
            for (features_np, card_idx, draw_idx, step_reward) in plays_p1:
                total_reward = episode_booster * reward_p1 + step_reward
                replay_buffer.append((features_np, card_idx, draw_idx, total_reward))
                if ddebug:
                    p1cnt+=1
                    print(f"P1 {p1cnt} - {episode_booster} * {reward_p1} + {step_reward} = {total_reward}")
            for (features_np, card_idx, draw_idx, step_reward) in plays_p2:
                total_reward = episode_booster * reward_p2 + step_reward
                replay_buffer.append((features_np, card_idx, draw_idx, total_reward))

    # Final mean reward is the average over plays - approximate over P1 and P2
    mean_reward=1.0*mean_reward/play_cnt
    
    if play_cnt>200:
        print(f"Plays: {play_cnt} in episode {episode}")

    # Train
    if episode % train_every == 0 and len(replay_buffer) >= batch_size:
        for _ in range(batch_cnt):
            minibatch = random.sample(replay_buffer, batch_size)
    
            # Unpack minibatch into separate lists
            states_b, actions_b, draws_b, rewards_b = zip(*minibatch)

            # Convert lists to tensors
            states_np = np.array(states_b)  # Convert list of arrays → single array
            states_t = torch.tensor(states_np, dtype=torch.float32)
    
            # Convert to tensors in batch
            # states_t = torch.tensor(states_b, dtype=torch.float32)  # Shape: [batch_size, state_size]
            actions_t = torch.tensor(actions_b, dtype=torch.long)   # Shape: [batch_size]
            draws_t = torch.tensor(draws_b, dtype=torch.long)       # Shape: [batch_size]
            rewards_t = torch.tensor(rewards_b, dtype=torch.float32)  # Shape: [batch_size]
    
            # Forward pass in batch
            card_logits_b, draw_logits_b, values_b = model(states_t)  # Each output shape: [batch_size, num_actions/draws]
    
            # Compute log probs for card actions
            card_probs_b = torch.softmax(card_logits_b, dim=1)
            log_card_probs_b = torch.log(card_probs_b + 1e-8)
            selected_log_card_probs = log_card_probs_b[range(batch_size), actions_t]
    
            # Compute log probs for draws
            draw_probs_b = torch.softmax(draw_logits_b, dim=1)
            log_draw_probs_b = torch.log(draw_probs_b + 1e-8)
            selected_log_draw_probs = log_draw_probs_b[range(batch_size), draws_t]
    
            # Compute advantage
            advantages = rewards_t - values_b.squeeze(1)  # Shape: [batch_size]
    
            # Losses
            critic_loss = advantages.pow(2).mean()
            actor_loss_card = -(selected_log_card_probs * advantages).mean()
            actor_loss_draw = -(selected_log_draw_probs * advantages).mean()
    
            total_loss = critic_loss + actor_loss_card + actor_loss_draw
    
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

    all_rewards.append(reward_p1)
    all_rewards.append(reward_p2)    
    mean_rewards.append(mean_reward)

    if episode % 1000 == 0:
        print("\n=== Step Rule Firing Counts ===")
        for rule, count in sorted(rule_counter.items(), key=lambda x: -x[1]):
            print(f"{rule:<30}: {count}")    
    
    if episode % 200 == 0:
        avg_score = np.mean(all_rewards[-2000:]) if len(all_rewards) >= 2000 else np.mean(all_rewards)
        print(f"Episode {episode}, Average Reward Last {min(len(all_rewards), 1000)}: {avg_score:.2f}, eps={epsilon:.4f}")
        pd.Series(all_rewards).to_csv(file_name, index=False, header=False)

Episode 200, Average Reward Last 400: -1.21, eps=0.3480
Episode 400, Average Reward Last 800: -1.25, eps=0.3458
Episode 600, Average Reward Last 1000: -1.22, eps=0.3437
Episode 800, Average Reward Last 1000: -1.50, eps=0.3416

=== Step Rule Firing Counts ===
draw_to_tgt                   : 6056
good_exp_1                    : 4837
too_few_pts                   : 3784
good_exp                      : 3060
blocked_7                     : 2663
good_low_val                  : 2405
exp_was_live                  : 1626
next_value                    : 1494
lower_val_avail               : 831
bad_bigger_val                : 831
exp_small_deck                : 768
bad_X                         : 450
Episode 1000, Average Reward Last 1000: -1.83, eps=0.3395
Episode 1200, Average Reward Last 1000: -1.98, eps=0.3374
Episode 1400, Average Reward Last 1000: -2.25, eps=0.3354
Episode 1600, Average Reward Last 1000: -2.62, eps=0.3334
Episode 1800, Average Reward Last 1000: -2.69, eps=0.3314

=== Step R

In [80]:
torch.save(model.state_dict(), 'lc_model_'+fv+'.pt')

In [93]:
def play_bot(epsilon=0.0):
    state = env.reset()
    done = False
    play_cnt = 0

    while not done:
        play_cnt += 1
        current_player = state['current_player']

        print(f"\n=== Turn {play_cnt} — {current_player} ===")

        actions, draws = env.get_legal_actions(current_player)
        valid_draws = [d for d in draws if d == 'deck' or (d in env.center_piles and env.center_piles[d])]

        if not actions or not valid_draws:
            print("No legal actions or draws. Ending game.")
            break

        if current_player == 'P2':
            # ===== HUMAN TURN =====
            hand = state['hands'][current_player]
            exped = env.expeditions[current_player]
            center = env.center_piles
            opp_exped = env.expeditions['P1']

            exped_str = " | ".join(f"{color}:{exped.get(color, [])}" for color in COLORS)
            opp_exped_str = " | ".join(f"{color}:{opp_exped.get(color, [])}" for color in COLORS)
            center_str = " | ".join(f"{color}:{center[color]}" for color in COLORS)

            print(f"Your hand: {hand}")
            print(f"Your expeditions: {exped_str}")
            print(f"Center piles: {center_str}")
            print(f"Opponent expeditions: {opp_exped_str}")
            print(f"Valid draws: {valid_draws}")

            user_input = input("Enter move as (E/C) CARD DRAW (e.g., E B3 D): ").strip().upper()
            try:
                move_type, card_str, draw_choice = user_input.split()
                assert move_type in ['E', 'C']
                assert any(card_str == card for _, card in actions), "Invalid card"
                assert draw_choice in ['D'] + COLORS, "Invalid draw"
            except Exception as e:
                print("Invalid input. Try again.")
                continue

            chosen_action = ('expedition', card_str) if move_type == 'E' else ('center', card_str)
            chosen_draw = 'deck' if draw_choice == 'D' else draw_choice

        else:
            # ===== MODEL TURN =====
            features_np = extract_features(state)
            features = torch.tensor(features_np, dtype=torch.float32)
            card_logits, draw_logits, value = model(features)

            # Card action selection (no change)
            card_probs = torch.softmax(card_logits[:len(actions)], dim=0)
            card_dist = torch.distributions.Categorical(card_probs)
            card_idx = card_dist.sample().item()
            chosen_action = actions[card_idx]

            # Filter valid draws based on chosen_action (if it's a center discard)
            discard_color = None
            if chosen_action[0] == 'center':
                discard_color = chosen_action[1][0]
            
            filtered_draws = [
                d for d in valid_draws if d != discard_color
            ]
            if not filtered_draws:
                # Failsafe: fallback to deck
                filtered_draws = ['deck']

            valid_draws=filtered_draws
            
            # Draw choice selection (FIXED)
            if random.random() < epsilon:
                chosen_draw = random.choice(valid_draws)
            else:
                # Map valid draws to their indices in logits
                draw_indices_in_logits = [draw_to_index[d] for d in valid_draws]
                draw_logits_filtered = draw_logits[draw_indices_in_logits]
                draw_probs = torch.softmax(draw_logits_filtered, dim=0)
                draw_dist = torch.distributions.Categorical(draw_probs)
                draw_idx = draw_dist.sample().item()
                chosen_draw = valid_draws[draw_idx]

            opp_hand = state['hands'][current_player]
            print(f"P1 plays: {chosen_action} | Draws: {chosen_draw} -- holding {opp_hand}")

        # Step and advance
        next_state, reward, done = env.step(chosen_action, chosen_draw)
        state = next_state

        if done:
            p1_score = env.compute_score('P1')
            p2_score = env.compute_score('P2')
            print(f"\n=== Game Over ===")
            print(f"Final Score — P1: {p1_score} | P2: {p2_score}")

In [87]:
model.load_state_dict(torch.load('lc_model_v1B_2.pt'))
model.eval()  # Important: sets model to evaluation mode (no dropout etc.)

ActorCritic(
  (fc1): Linear(in_features=43, out_features=96, bias=True)
  (fc2): Linear(in_features=96, out_features=32, bias=True)
  (dropout): Dropout(p=0.15, inplace=False)
  (policy_action_head): Linear(in_features=32, out_features=18, bias=True)
  (policy_draw_head): Linear(in_features=32, out_features=4, bias=True)
  (value_head): Linear(in_features=32, out_features=1, bias=True)
)

In [95]:
play_bot(0.00)


=== Turn 1 — P1 ===
P1 plays: ('center', 'B3') | Draws: deck -- holding ['B3', 'B4', 'B6']

=== Turn 2 — P2 ===
Your hand: ['R2', 'R3', 'R6']
Your expeditions: R:[] | B:[] | G:[]
Center piles: R:[] | B:['B3'] | G:[]
Opponent expeditions: R:[] | B:[] | G:[]
Valid draws: ['deck', 'B']


KeyboardInterrupt: Interrupted by user